<a href="https://colab.research.google.com/github/RHITHIKAG/DLCV/blob/main/DLcvAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement MLP from scratch & replace the backpropagation algorithm with any of the latest optimization algorithms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.W1 = np.random.randn(self.input_size, self.hidden_size) / np.sqrt(self.input_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) / np.sqrt(self.hidden_size)
        self.b2 = np.zeros((1, self.output_size))
        self.loss_history = []
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, X):
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = self.relu(Z1)
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = self.softmax(Z2)
        return A1, A2
    
    def backward(self, X, Y, A1, A2):
        dZ2 = A2 - Y
        dW2 = np.dot(A1.T, dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * (A1 > 0)
        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)
        return dW1, db1, dW2, db2
    
    def fit(self, X, Y, epochs=100):
        for i in range(epochs):
            A1, A2 = self.forward(X)
            dW1, db1, dW2, db2 = self.backward(X, Y, A1, A2)
            self.W1 -= self.learning_rate * dW1
            self.b1 -= self.learning_rate * db1
            self.W2 -= self.learning_rate * dW2
            self.b2 -= self.learning_rate * db2
            loss = self.compute_loss(X, Y)
            self.loss_history.append(loss)
    
    def predict(self, X):
        _, A2 = self.forward(X)
        return np.argmax(A2, axis=1)
    
    def compute_loss(self, X, Y):
        _, A2 = self.forward(X)
        num_samples = X.shape[0]
        log_likelihood = -np.log(A2[range(num_samples), Y.argmax(axis=1)])
        loss = np.sum(log_likelihood) / num_samples
        return loss


# Load data
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
Y = iris.target

# Convert target to one-hot encoding
num_classes = len(np.unique(Y))
Y_one_hot = np.eye(num_classes)[Y]

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_one_hot, test_size=0.2, random_state=42)

# Create MLP object and train the model
mlp = MLP(input_size=X_train.shape[1], hidden_size=64, output_size=num_classes)
mlp.fit(X_train, Y_train, epochs=100)

# Evaluate the model on test data
accuracy = np.mean(np.argmax(Y_test, axis=1) == mlp.predict(X_test))
print(f"Test accuracy: {accuracy:.2f}")



Test accuracy: 0.30
